In [24]:
#Alphabet Sign Language Data Collection using MediaPipe and OpenCV

import cv2
import mediapipe as mp
import pandas as pd

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

data = []
labels = []

current_label = "R"  # Change this when collecting other alphabets

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract 21 landmark points
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            data.append(landmarks)
            labels.append(current_label)

    cv2.imshow("Collecting Data", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to stop
        break

cap.release()
cv2.destroyAllWindows()

# Save collected dataset
df = pd.DataFrame(data)
df['label'] = labels
df.to_csv("hand_sign_data_R.csv", index=False)


c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [2]:
import pandas as pd
import glob
import os

# Folder containing all your datasets
DATA_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\AlphabetSignLanguages\FSL-alphabet-signs"

# Load all CSV files in the folder
all_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))

# Read and concatenate all into one dataframe
df_list = []
for file in all_files:
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)

# Combine into one dataset
final_df = pd.concat(df_list, ignore_index=True)

# Save as one combined dataset
final_csv = os.path.join(DATA_DIR, "Alphabetical_hand_sign_data.csv")
final_df.to_csv(final_csv, index=False)

print("✅ Combined dataset saved at:", final_csv)
print("Dataset shape:", final_df.shape)
print("Labels available:", final_df['label'].unique())

✅ Combined dataset saved at: C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\AlphabetSignLanguages\FSL-alphabet-signs\Alphabetical_hand_sign_data.csv
Dataset shape: (51319, 64)
Labels available: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle 

# Load final dataset
file_path = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\AlphabetSignLanguages\FSL-alphabet-signs\Alphabetical_hand_sign_data.csv"
df = pd.read_csv(file_path)

# Features (X) and labels (y)
X = df.drop("label", axis=1)
y = df["label"]

# Split into train/test (stratify keeps A–Z balanced)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Evaluate
print("Accuracy:", model.score(X_test, y_test))
print("Classes learned:", model.classes_)  # should show A–Z

# Save trained model
with open("sign_model.pkl", "wb") as f:
    pickle.dump(model, f)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\JamJayDatuin\\Documents\\Machine Learning Projects\\TrainingAI-Models\\alphabet-signs\\Alphabetical_hand_sign_data.csv'

In [27]:
import cv2
import mediapipe as mp
import pickle
import numpy as np
from collections import deque

# Load trained model
with open("sign_model.pkl", "rb") as f:
    model = pickle.load(f)

# Mediapipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)

# For smoothing (keep last 10 predictions)
history = deque(maxlen=10)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    predicted_label = "No hands detected"

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract landmarks (21 points × 3 coords)
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            # Convert to NumPy array for prediction
            landmarks = np.array(landmarks).reshape(1, -1)

            try:
                # Predict class
                prediction = model.predict(landmarks)
                predicted_label = prediction[0]
            except Exception as e:
                predicted_label = f"Error: {e}"

    # Store prediction history
    history.append(predicted_label)

    # Smooth result → most common label in history
    if history:
        predicted_label = max(set(history), key=history.count)

    # Show result on frame
    cv2.putText(frame, f"Predicted: {predicted_label}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Sign Language Recognition", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Ple